<a href="https://colab.research.google.com/github/JaiminPatel23/AI/blob/master/ICA%20PROJECT/J035_edx_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DEGREES**

In [ ]:
import csv
import sys

from util import Node, StackFrontier, QueueFrontier
import timer
# Maps names to a set of corresponding person_ids
names = {}

# Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}


def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass


def main():
    if len(sys.argv) > 2:
        sys.exit("Usage: python degrees.py [directory]")
    directory = sys.argv[1] if len(sys.argv) == 2 else "large"

    # Load data from files into memory
    print("Loading data...")
    load_data(directory)
    print("Data loaded.")

    source = person_id_for_name(input("Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Name: "))
    if target is None:
        sys.exit("Person not found.")

    t=timer.Timer()

    t.start()
    path = shortest_path(source, target)
    t.stop()

    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"{degrees} degrees of separation.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")


def shortest_path(source, target):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.
    If no possible path, returns None.
    """

    # Initialize frontier to just the starting position
    start = Node(state=source, parent=None, action=None)
    frontier = StackFrontier()
    frontier.add(start)

    # Initialize an empty explored set
    explored = set()

    # Keep looping until solution found
    while True:

        # If nothing left in frontier, then no path
        if frontier.empty():
            raise Exception("no solution")

        # Choose a node from the frontier
        node = frontier.remove()

        # If node is the goal, then we have a solution
        if node.state == target:
            moviess = []
            actors = []
            while node.parent is not None:
                moviess.append(node.action)
                actors.append(node.state)
                node = node.parent
            moviess.reverse()
            actors.reverse()
            solution = []
            for i in range(len(moviess)):
                solution.append((moviess[i], actors[i]))

            return solution

        # Mark node as explored
        explored.add(node.state)

        # Add neighbors to frontier
        for movie_id, person_id in neighbors_for_person(node.state):
            if not frontier.contains_state(person_id) and person_id not in explored:
                child = Node(state=person_id, parent=node, action=movie_id)
                frontier.add(child)


def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]


def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors


if __name__ == "__main__":
    main()

**TIC-TAC-TOE**

In [1]:
"""
Tic Tac Toe Player
"""

import math
import copy

X = "X"
O = "O"
EMPTY = None


def initial_state():
    """
    Returns starting state of the board.
    """
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]]


def player(board):
    """
    Returns player who has the next turn on a board.
    """
    num_of_x = 0
    num_of_o = 0

    # calculate the quantity of X and O
    for i in range(3):
        for j in range(3):
            if board[i][j] == X:
                num_of_x += 1
            elif board[i][j] == O:
                num_of_o += 1

    if num_of_x > num_of_o:
        return(O)
    else:
        return(X)


def actions(board):
    """
    Returns set of all possible actions (i, j) available on the board.
    """
    actions = set()

    for i in range(3):
        for j in range(3):
            if board[i][j] == EMPTY:
                actions.add((i, j))
    
    return(actions)


def result(board, action):
    """
    Returns the board that results from making move (i, j) on the board.
    """
    new_board = copy.deepcopy(board)
    move = player(board)
    if new_board[action[0]][action[1]] == EMPTY:
        new_board[action[0]][action[1]] = move
        return(new_board)
    else:
        raise NameError('Not valid action')


def winner(board):
    """
    Returns the winner of the game, if there is one.
    """
    for i in range(3):
        for j in range(3):
            if board[i][j] != EMPTY:
                #check diagonals
                if i == 0 and j == 0:
                    if board[i][j] == board[i+1][j+1] == board[i+2][j+2]:
                        return(board[i][j])
                
                if i == 0 and j == 2:
                    if board[i][j] == board[i+1][j-1] == board[i+2][j-2]:
                       return(board[i][j])
                #check horizontal rows
                try:
                    if board[i][j] == board[i][j+1] == board[i][j+2]:
                        return(board[i][j])

                except IndexError:
                    pass
                #check vertical rows     
                try:
                    if board[i][j] == board[i+1][j] == board[i+2][j]:
                        return(board[i][j])     
                except IndexError:
                    pass
    return(None)   


def terminal(board):
    """
    Returns True if game is over, False otherwise.
    """
    if winner(board) == None:
        for i in range(3):
            for j in range(3):
                if board[i][j] == None:
                    return(False)
    return(True)




def utility(board):
    """
    Returns 1 if X has won the game, -1 if O has won, 0 otherwise.
    """
    tmp = winner(board)
    if tmp == X:
        return(1)
    elif tmp == O:
        return(-1)
    else:
        return(0)


def minimax(board):
    """
    Returns the optimal action for the current player on the board.
    """
    
    def min_value(board):
        v = 2
        if terminal(board):
            return(utility(board))
        
        for move in actions(board):
            v = min(v, max_value(result(board, move)))
            if v == -1:
                break
        
        return v

    def max_value(board):
        v = -2
        if terminal(board):
            return(utility(board))
        
        for move in actions(board):
            v = max(v, min_value(result(board, move)))
            if v == 1:
                break
        
        return v


    if terminal(board):
        return None 

    moves = actions(board)
    
    if player(board) == X:
        for move in moves:
            if min_value(result(board, move)) == 1:
                return move
        for move in moves:    
            if min_value(result(board, move)) == 0:
                return move

    elif player(board) == O:
        for move in moves:
            if max_value(result(board, move)) == -1:
                return move
        for move in moves:    
            if max_value(result(board, move)) == 0:
                return move